In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K

In [ ]:
import tensorflow as tf
tf.__version__
tf.test.gpu_device_name()

In [ ]:
K.clear_session()

data_entrenamiento = './data/train'
data_validacion = './data/validation'

In [ ]:
epocas = 20 #numero de veces q se va a iterar sobre el set de datos.
width_shape = 224
height_shape = 224 # tamano al cual vamos a procesar las imagenes
batch_size = 32 #cantidad de imagenes que enviamos a procesar en cada uno de los pasos
clases=2 #tipo de imagenes q vamos a enviar
lr=0.0005 #determina el tamano de los ajustes que ara nuestra red neuronal

PRE PROCESAMIENTO DE IMAGENES

In [ ]:
train_datagen = ImageDataGenerator()

valid_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    class_mode='categorical')
validation_generator = valid_datagen.flow_from_directory(
    data_validacion,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
nb_train_samples = 10682
nb_validation_samples = 3562

MODELO VGG16

In [ ]:
image_input = Input(shape=(width_shape, height_shape, 3))
model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')
last_layer = model.layers[-1].output
out = Dense(clases, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)
custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
custom_vgg_model.summary()

ENTRENAMIENTO MODELO VGG16

In [ ]:
model_history = custom_vgg_model.fit_generator(
    train_generator,
    epochs=epocas,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size)

In [ ]:
custom_vgg_model.save("./model/model_VGG16-20.h5")

Modelo Resnet50

In [ ]:
from keras.applications.resnet import ResNet50
image_input = Input(shape=(width_shape, height_shape, 3))
m_Resnet50 = ResNet50(input_tensor=image_input, include_top=False, weights='imagenet')

last_layer = m_Resnet50.layers[-1].output
x = Flatten(name='flatten')(last_layer)
out = Dense(clases, activation='softmax', name='output')(x)
custom_model = Model(image_input, out)
custom_model.summary()

custom_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [ ]:
model_history = custom_model.fit_generator(
    train_generator,
    epochs=epocas,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size)

In [ ]:
custom_model.save("./model/model_Resnet50-20.h5")

Transfer Learning modelo VGG19 - fine tune

In [ ]:
from keras.applications.vgg19 import VGG19
image_input = Input(shape=(width_shape, height_shape, 3))
m_VGG19 = VGG19(input_tensor=image_input, include_top=False,weights='imagenet')
m_VGG19.summary()
last_layer = m_VGG19.layers[-1].output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x=Dropout(0.3)(x)
x = Dense(128, activation='relu', name='fc2')(x)
x=Dropout(0.3)(x)
out = Dense(clases, activation='softmax', name='output')(x)
custom_model = Model(image_input, out)
custom_model.summary()
custom_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [ ]:
model_history = custom_model.fit_generator(
    train_generator,
    epochs=epocas,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size)

In [ ]:
custom_model.save("./model/model_VGG19-20.h5")

Modelo Mobilenet - Entrenamiento de toda la red

In [ ]:
from keras.applications.mobilenet import MobileNet
image_input = Input(shape=(width_shape, height_shape, 3))
m_MobileNet = MobileNet(input_tensor=image_input, include_top=False,weights='imagenet')
m_MobileNet.summary()
last_layer = m_MobileNet.layers[-1].output
x= Flatten(name='flatten')(last_layer)
out = Dense(clases, activation='softmax', name='output')(x)
custom_model = Model(image_input, out)
custom_model.summary()
custom_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [ ]:
model_history = custom_model.fit_generator(
    train_generator,
    epochs=epocas,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size)

In [ ]:
custom_model.save("./model/model_Mobilenet-20.h5")

Gráficas de entrenamiento y validación (accuracy - loss)

In [ ]:
def plotTraining(hist, epochs, typeData):
    if typeData == "loss":
        plt.figure(1, figsize=(10, 5))
        yc = hist.history['loss']
        xc = range(epochs)
        plt.ylabel('Loss', fontsize=24)
        plt.plot(xc, yc, '-r', label='Loss Training')
    if typeData == "accuracy":
        plt.figure(2, figsize=(10, 5))
        yc = hist.history['accuracy']
        for i in range(0, len(yc)):
            yc[i] = 100 * yc[i]
        xc = range(epochs)
        plt.ylabel('Accuracy (%)', fontsize=24)
        plt.plot(xc, yc, '-r', label='Accuracy Training')
    if typeData == "val_loss":
        plt.figure(1, figsize=(10, 5))
        yc = hist.history['val_loss']
        xc = range(epochs)
        plt.ylabel('Loss', fontsize=24)
        plt.plot(xc, yc, '--b', label='Loss Validate')
    if typeData == "val_accuracy":
        plt.figure(2, figsize=(10, 5))
        yc = hist.history['val_accuracy']
        for i in range(0, len(yc)):
            yc[i] = 100 * yc[i]
        xc = range(epochs)
        plt.ylabel('Accuracy (%)', fontsize=24)
        plt.plot(xc, yc, '--b', label='Training Validate')

    plt.rc('xtick', labelsize=24)
    plt.rc('ytick', labelsize=24)
    plt.rc('legend', fontsize=18)
    plt.legend()
    plt.xlabel('Number of Epochs', fontsize=24)
    plt.grid(True)

plotTraining(model_history,epocas,"loss")
plotTraining(model_history,epocas,"accuracy")
plotTraining(model_history,epocas,"val_loss")
plotTraining(model_history,epocas,"val_accuracy")